# Imports

In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 30.2 MB/s eta 0:00:00


In [ ]:
# install chromium, its driver, and selenium
!apt update
!apt install libu2f-udev libvulkan1
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!wget https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/linux64/chromedriver-linux64.zip
!unzip -j chromedriver-linux64.zip chromedriver-linux64/chromedriver -d /usr/local/bin/
!pip install selenium chromedriver_autoinstaller

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,606 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,230 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu 

In [ ]:
import json
import re
import time

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import (NoSuchElementException,
                                        StaleElementReferenceException,
                                        TimeoutException)
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [ ]:
# set options to be headless, ..
from selenium import webdriver
import chromedriver_autoinstaller
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
chromedriver_autoinstaller.install()

'/usr/local/lib/python3.11/dist-packages/chromedriver_autoinstaller/133/chromedriver'

# Загрузка ссылок на людей


In [ ]:
def get_links_with_selenium(base_url):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    wd = webdriver.Chrome(options=options)
    wd.get(base_url)

    all_links = set()
    last_height = wd.execute_script("return document.body.scrollHeight")

    while True:
        # Прокрутка вниз
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Ожидание загрузки новых элементов
        time.sleep(2)

        # Парсинг текущей страницы
        soup = BeautifulSoup(wd.page_source, 'html.parser')
        diaries_list = soup.find('ul', class_='diaries-list')

        if diaries_list:
            for item in diaries_list.find_all('li', class_='diaries-list__item'):
                link = item.find('div', class_='diaries-list__content').find('h3', class_='diaries-list__title').find('a', href=True)
                if link:
                    statuses = item.find('ul', class_='tags-list')
                    if statuses:
                        status = statuses.find_all('li', class_='tags-list__item')
                        for status_item in status:
                            closed_text = status_item.find('a', class_='statuses-list__link')
                            if closed_text and closed_text.text.strip() != 'текст закрыт':
                              # добавить сюда еще исключения на "нужен текст"
                                all_links.add('https://corpus.prozhito.org' + link['href'])

        # Проверка, достигнут ли конец страницы
        new_height = wd.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # Попытка прокрутить еще раз для уверенности
            wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = wd.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
        last_height = new_height

        print(f"Найдено ссылок: {len(all_links)}")

    wd.quit()
    return list(all_links)

def main():
    base_url = 'https://corpus.prozhito.org/persons?date=%221900-01-01%22&dateTop=%222024-12-31%22&textType=%5B%22all%22%5D'
    all_links = get_links_with_selenium(base_url)

    print(f"Всего найдено уникальных ссылок: {len(all_links)}")

    # Сохраняем ссылки в файл
    with open('prozhito_links.txt', 'w', encoding='utf-8') as f:
        for link in all_links:
            f.write(link + '\n')

    print("Ссылки сохранены в файл 'prozhito_links.txt'")

if __name__ == "__main__":
    main()

Найдено ссылок: 118
Найдено ссылок: 277
Найдено ссылок: 596
Найдено ссылок: 1235
Найдено ссылок: 2512
Найдено ссылок: 2814
Всего найдено уникальных ссылок: 2814
Ссылки сохранены в файл 'prozhito_links.txt'


# Загрузка ссылок на записи дневников

In [ ]:
def get_diary_links_with_selenium(person_url):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    wd = webdriver.Chrome(options=options)
    wd.get(person_url)

    diary_links = set()  # Множество для уникальных ссылок

    last_height = wd.execute_script("return document.body.scrollHeight")

    while True:
        # Прокрутка страницы вниз
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Ожидание загрузки новых элементов
        time.sleep(2)

        # Парсинг текущей страницы
        soup = BeautifulSoup(wd.page_source, 'html.parser')

        # Поиск нужного блока с дневниками
        diary_section = soup.find('div', class_='notes_list')

        if diary_section:
          # Найти первую ссылку на дневниковую запись
          first_link = diary_section.find('a', class_='diary-info__rarety clearfix')
          if first_link:
              diary_links.add('https://corpus.prozhito.org' + first_link['href'])
              break  # Прекращаем цикл после первой найденной ссылки

        # Проверка, достигнут ли конец страницы
        new_height = wd.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # Попробуем прокрутить еще раз на всякий случай
            wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = wd.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
        last_height = new_height

        print(f"Найдено ссылок: {len(diary_links)}")

    wd.quit()
    return list(diary_links)

def main():
    input_file = 'prozhito_links (2).txt'
    output_file = 'diary_links.txt'
    processed_file = 'processed_prozhito_links.txt'  # Файл с уже обработанными ссылками

    num_links_to_download = 100  # Количество ссылок для скачивания

    # Загружаем уже скачанные ссылки
    try:
        with open(output_file, 'r', encoding='utf-8') as f:
            downloaded_links = set(line.strip() for line in f if line.strip())
    except FileNotFoundError:
        downloaded_links = set()

    # Загружаем уже обработанные ссылки
    try:
        with open(processed_file, 'r', encoding='utf-8') as f:
            processed_links = set(line.strip() for line in f if line.strip())
    except FileNotFoundError:
        processed_links = set()

    with open(input_file, 'r', encoding='utf-8') as f:
        person_urls = [line.strip() for line in f if line.strip()]

    added_count = 0  # Счетчик добавленных ссылок

    for person_url in person_urls:
        if added_count >= num_links_to_download:
            break  # Достигли нужного количества, выходим

        if person_url in processed_links:
            print(f"Ссылка уже обрабатывалась ранее: {person_url}")
            continue  # Пропускаем уже обработанные ссылки

        print(f"Обрабатывается: {person_url}")
        diary_links = get_diary_links_with_selenium(person_url)  # Получаем ссылки

        if diary_links:
            diary_link = diary_links[0]  # Берем первую ссылку

            if diary_link not in downloaded_links:
                with open(output_file, 'a', encoding='utf-8') as f:
                    f.write(diary_link + '\n')
                print(f"Добавлена новая ссылка: {diary_link}")
                added_count += 1  # Увеличиваем счетчик добавленных ссылок
            else:
                print(f"Ссылка уже была скачана: {diary_link}")

        # Сохраняем обработанную ссылку, даже если не нашли дневниковые записи
        with open(processed_file, 'a', encoding='utf-8') as f:
            f.write(person_url + '\n')

    print(f"Добавлено {added_count} новых ссылок.")


if __name__ == "__main__":
    main()


Ссылка уже обрабатывалась ранее: https://corpus.prozhito.org/person/2882
Ссылка уже обрабатывалась ранее: https://corpus.prozhito.org/person/1953
Ссылка уже обрабатывалась ранее: https://corpus.prozhito.org/person/1333
Ссылка уже обрабатывалась ранее: https://corpus.prozhito.org/person/9723
Ссылка уже обрабатывалась ранее: https://corpus.prozhito.org/person/1619
Ссылка уже обрабатывалась ранее: https://corpus.prozhito.org/person/110
Ссылка уже обрабатывалась ранее: https://corpus.prozhito.org/person/1962
Ссылка уже обрабатывалась ранее: https://corpus.prozhito.org/person/5638
Ссылка уже обрабатывалась ранее: https://corpus.prozhito.org/person/5436
Ссылка уже обрабатывалась ранее: https://corpus.prozhito.org/person/1757
Ссылка уже обрабатывалась ранее: https://corpus.prozhito.org/person/5576
Ссылка уже обрабатывалась ранее: https://corpus.prozhito.org/person/3404
Ссылка уже обрабатывалась ранее: https://corpus.prozhito.org/person/549
Ссылка уже обрабатывалась ранее: https://corpus.prozh

# Получение записей через API

Логика кода:

На html странице ищем корректную ссылку и максимальное значение offset

Корректную ссылку преобразуем в API ссылку

Получаем номер сессии с API ссылки

Получаем тексты и года с помощью номера сессии и оффсетов (генерируем тексты до момента достижения максимального оффсета)

Сохраняем

Если есть

div class load-more-inner

То будем через API тексты брать

Иначе - это одностраничный динамический сайт


## получаем тексты

In [ ]:
'https://corpus.prozhito.org/notes?diaries=%5B4236%5D&diaryTypes=%5B1%5D&offset=0' == 'https://corpus.prozhito.org/notes?diaries=%5B4236%5D&diaryTypes=%5B1%5D&offset=0'

True

для этого нужен номер сессии конкретного дневника

In [ ]:
import requests

def fetch_diary_entries(offset=0):
    url = 'https://corpus.prozhito.org/api/notes/search'
    params = {
        'search_type': 'diaries',
        'offset': offset,
        'searchNotesSession': '17388373092b4c1c219a54adab4b6ebd19fceded6f'  # Замените на актуальное значение, если необходимо
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data  # Возвращает данные в формате JSON
    else:
        print(f"Ошибка: {response.status_code}")
        return None

# Пример использования
diary_entries = fetch_diary_entries(offset=25)  # Замените 50 на нужное вам смещение
if diary_entries:
    print(diary_entries)  # Вывод полученных данных
    print(len(diary_entries['data']['notes']))
    for el in diary_entries['data']['notes']:
      print(el['text'])

with open('response', 'w', encoding='utf-8') as f:
    json.dump(diary_entries, f, ensure_ascii=False, indent=2)


{'status': 1, 'data': {'notes': [{'id': 365949, 'diary': 2180, 'text': '<p>Бились за хлебозавод, от которого остались груды кирпича и жести. Еще с утра, как затарахтела кухня, начался минометный огонь редкой точности. Одна «кухонная» мина, упавшая чуть ли не в суп, помешала допить водку, ранив двух человек, и так целый день. Соседний блиндаж разметало снарядом. Бои в воздухе как-то не ладятся. Только немцы сделают свое дело — появляются наши. На земле скрежещут зубами.</p>', 'date': '1943-01-13', 'julian_date': None, 'dateTop': '0000-00-00', 'notDated': 0, 'julian_calendar': 0, 'user': 14, 'createdDate': 1522836079, 'pictures': 1, 'age': 19, 'comments': [], 'hidden_note': False, 'snippet': '', 'tags': [], 'mentioned_people': [], 'person': {'id': 2170, 'firstName': 'Олег', 'lastName': 'Плиндов', 'thirdName': 'Назарович', 'birthDay': '14 марта 1923', 'deathDay': '', 'birthDay2': '0000-00-00', 'deathDay2': '0000-00-00', 'ageAround': 0, 'info': '', 'edition': '', 'additional_info': '', 'fo

## Получение searchNotesSession

In [ ]:
'172842164639cc11d8f2bb5b4709bbbed8c704a961' == '172842164639cc11d8f2bb5b4709bbbed8c704a961'

True

Для одного человека  и одного днекника -- этот параметр одинаковый

Чтобы ее получить нужна API ссылка

In [ ]:
def fetch_diary_entries(offset=0):
    url = 'https://corpus.prozhito.org/api/notes/search?search_type=diaries&diaryTypes=%5B1%5D&diaries=%5B2180%5D'

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        return data['data']['searchNotesSession']  # Возвращает данные в формате JSON
    else:
        print(f"Ошибка: {response.status_code}")
        return None

# Пример использования
diary_entries = fetch_diary_entries()  # Замените 50 на нужное вам смещение
if diary_entries:
    print(diary_entries)  # Вывод полученных данных

17388373092b4c1c219a54adab4b6ebd19fceded6f


## Получение корректной ссылки

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_dynamic_link(url):

    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=options)
    driver.get(url)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Находим элемент с классом load-more-inner
    load_more_inner = soup.find('div', class_ = 'load-more-inner')

    if load_more_inner:
        a_el = load_more_inner.select_one('a')
        # Извлекаем href из элемента
        full_link = a_el['href']
        # Преобразуем относительный URL в абсолютный
        full_link = f'https://corpus.prozhito.org{full_link}'
        return full_link

    return None

# Пример использования
url = 'https://corpus.prozhito.org/notes?date=%221964-01-01%22&diaries=%5B1766%5D'
dynamic_link = get_dynamic_link(url)

if dynamic_link:
    print(f"Найдена ссылка: {dynamic_link}")
else:
    print("Не удалось найти ссылку")


Не удалось найти ссылку


## Получение максимального параметра offset

Для этого нужна html ссылка

In [ ]:
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

def get_max_offset(url):
    # Настройки для headless браузера
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    # Инициализация веб-драйвера
    driver = webdriver.Chrome(options=options)
    driver.get(url)

    # Получение страницы и парсинг с помощью BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Находим элемент с классом load-more-inner
    load_more_inner = soup.find('div', class_='load-more-inner')

    if load_more_inner:
        # Находим все элементы <a> внутри load-more-inner и берем последний
        a_elements = load_more_inner.find_all('a')
        if a_elements:
            last_a = a_elements[-1]  # Последний элемент <a>
            href = last_a['href']

            # Извлечение параметра offset с помощью регулярного выражения
            match = re.search(r'offset=(\d+)', href)
            if match:
                max_offset = int(match.group(1))
                return max_offset

    return None

# Пример использования
url = 'https://corpus.prozhito.org/notes?diaryTypes=%5B1%5D&diaries=%5B2180%5D&'
max_offset = get_max_offset(url)

if max_offset is not None:
    print(f"Максимальное значение offset: {max_offset}")
else:
    print("Не удалось найти offset")


Максимальное значение offset: 25


## Преобразование корректной ссылки в API ссылку

In [ ]:
import re

def convert_url(url):
    # Определяем паттерн для URL
    pattern = r"https://corpus.prozhito.org/notes\?diaryTypes=(\%5B\d+\%5D)&diaries=(\%5B\d+\%5D)&?"

    # Используем регекс для нахождения нужных частей
    match = re.search(pattern, url)

    if match:
        diary_types = match.group(1)  # Извлекаем значение параметра diaryTypes
        diaries = match.group(2)  # Извлекаем значение параметра diaries

        # Формируем новую ссылку
        api_url = f"https://corpus.prozhito.org/api/notes/search?search_type=diaries&diaryTypes={diary_types}&diaries={diaries}"
        return api_url
    else:
        return None

# Пример использования
url = " https://corpus.prozhito.org/notes?diaryTypes=%5B1%5D&diaries=%5B2180%5D&"
api_url = convert_url(url)

if api_url:
    print("Converted URL:", api_url)
else:
    print("Invalid URL format")

# Пример для сравнения с оригинальной ссылкой
orig_link = 'https://corpus.prozhito.org/api/notes/search?search_type=diaries&diaryTypes=%5B1%5D&diaries=%5B2682%5D'

print(api_url == orig_link)  # Выводит True, если ссылки совпадают


Converted URL: https://corpus.prozhito.org/api/notes/search?search_type=diaries&diaryTypes=%5B1%5D&diaries=%5B2180%5D
False


## Общая функция

In [ ]:
def extract_text_from_span(item):
    text = ""
    # Проверяем сначала full_text
    full_text_span = item.select_one('div.notes-list__desc > div > div.full_text > span')
    if full_text_span:
        for element in full_text_span.children:
            if element.name == 'p':
                paragraph_text = element.get_text(strip=True)
                if paragraph_text:
                    text += paragraph_text + "\n"

    # Если full_text пуст, проверяем short_txt
    if not text:
        short_txt_span = item.select_one('div.notes-list__desc > div.short_txt > span')
        if short_txt_span:
            for element in short_txt_span.children:
                if element.name == 'p':
                    paragraph_text = element.get_text(strip=True)
                    if paragraph_text:
                        text += paragraph_text + "\n"

    return text.strip()

def scroll_to_bottom(driver):
    """Прокрутка страницы до конца."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(10)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def click_load_more(driver, max_attempts=1):
    """Находит и кликает кнопку 'Загрузить еще' с несколькими попытками."""
    for attempt in range(max_attempts):
        try:
            load_more_button = WebDriverWait(driver, 20).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, "button.btn-basic.red"))
            )
            driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
            driver.execute_script("arguments[0].click();", load_more_button)
            time.sleep(7)  # Увеличиваем время ожидания
            return True
        except (NoSuchElementException, TimeoutException):
            print(f"Попытка {attempt + 1}: Кнопка 'Загрузить еще' не найдена или не кликабельна.")
        except StaleElementReferenceException:
            print(f"Попытка {attempt + 1}: Элемент устарел, пробуем еще раз.")

        # Увеличиваем паузу перед следующей попыткой
        time.sleep(7)

    print("Не удалось нажать кнопку 'Загрузить еще' после нескольких попыток.")
    return False


def scrape_diary_entries(url, entry_limit=200):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=options)
    driver.get(url)

    entries = {}
    collected_texts = set()  # Множество для уникальных записей
    current_year = None
    total_entries = 0
    consecutive_failures = 0
    max_consecutive_failures = 1

    while total_entries < entry_limit:
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "notes-list"))
        )

        # Парсим текущую страницу
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        notes_list = soup.find('ul', class_='notes-list')

        if notes_list:
            new_entries = 0  # Сбрасываем перед каждым циклом
            for item in notes_list.find_all('li', class_='notes-list__item'):
                if total_entries >= entry_limit:
                    break

                year_div = item.find('p', class_='diary-content__year')
                if year_div:
                    current_year = year_div.text.strip()
                    if current_year not in entries:
                        entries[current_year] = []

                entry_text = extract_text_from_span(item)

                if current_year and entry_text and entry_text not in collected_texts:
                    entries[current_year].append(entry_text)
                    collected_texts.add(entry_text)
                    total_entries += 1
                    new_entries += 1  # Увеличиваем количество новых записей
                    print(f"Обработано записей: {total_entries}")

            # Прокрутка до конца страницы
            scroll_to_bottom(driver)
            time.sleep(20)  # Пауза для загрузки кнопки

            print(f"Текущее количество записей: {len(notes_list.find_all('li', class_='notes-list__item'))}")

            # Попытка нажать кнопку "Загрузить еще"
            if not click_load_more(driver):
                print("Не удалось нажать кнопку 'Загрузить еще'. Завершаем сбор данных.")
                break

            # Ждем загрузки новых записей
            time.sleep(10)  # Дайте время на загрузку новых данных
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CLASS_NAME, "notes-list"))
            )

            # Повторно парсим новые записи после загрузки
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            notes_list = soup.find('ul', class_='notes-list')

            # Подсчитываем количество записей после нажатия кнопки
            new_entries = len(notes_list.find_all('li', class_='notes-list__item'))
            print(f"Количество записей после загрузки: {new_entries}")


            # Подсчет новых записей после загрузки
            new_entries = 0  # Сбрасываем перед подсчетом
            if notes_list:
                for item in notes_list.find_all('li', class_='notes-list__item'):
                    if total_entries >= entry_limit:
                        break

                    year_div = item.find('p', class_='diary-content__year')
                    if year_div:
                        current_year = year_div.text.strip()
                        if current_year not in entries:
                            entries[current_year] = []

                    entry_text = extract_text_from_span(item)

                    if current_year and entry_text and entry_text not in collected_texts:
                        entries[current_year].append(entry_text)
                        collected_texts.add(entry_text)
                        total_entries += 1
                        new_entries += 1  # Увеличиваем количество новых записей

                print(f"Найдено новых записей: {new_entries}")  # Вывод количества новых записей

            # Проверка, появились ли новые записи после загрузки
            if new_entries == 0:
                consecutive_failures += 1
                if consecutive_failures >= max_consecutive_failures:
                    print(f"Не удалось загрузить новые записи после {max_consecutive_failures} попыток. Завершаем сбор данных.")
                    break
            else:
                consecutive_failures = 0  # Сбрасываем счетчик, если появились новые записи



        if total_entries >= entry_limit:
            break

    driver.quit()
    return entries, total_entries


In [ ]:
# Получение корректной ссылки
def get_dynamic_link(url):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=options)
    driver.get(url)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Находим элемент с классом load-more-inner
    load_more_inner = soup.find('div', class_='load-more-inner')

    if load_more_inner:
        a_el = load_more_inner.select_one('a')
        full_link = a_el['href']
        full_link = f'https://corpus.prozhito.org{full_link}'
        driver.quit()
        return full_link

    driver.quit()
    return None


# Преобразование корректной ссылки в API ссылку
def convert_url(url):
    pattern = r"https://corpus.prozhito.org/notes\?diaryTypes=(\%5B\d+\%5D)&diaries=(\%5B\d+\%5D)&?"
    match = re.search(pattern, url)

    if match:
        diary_types = match.group(1)
        diaries = match.group(2)
        api_url = f"https://corpus.prozhito.org/api/notes/search?search_type=diaries&diaryTypes={diary_types}&diaries={diaries}"
        return api_url
    else:
        return None


# Получение максимального значения offset
def get_max_offset(url):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=options)
    driver.get(url)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    load_more_inner = soup.find('div', class_='load-more-inner')

    if load_more_inner:
        a_elements = load_more_inner.find_all('a')
        if a_elements:
            last_a = a_elements[-1]
            href = last_a['href']
            match = re.search(r'offset=(\d+)', href)
            if match:
                max_offset = int(match.group(1))
                driver.quit()
                return max_offset

    driver.quit()
    return None


# Получение номера сессии API
def fetch_search_notes_session(url):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data['data']['searchNotesSession']
    else:
        print(f"Ошибка: {response.status_code}")
        return None


# Получение дневников через API
def fetch_diary_entries(offset=0, session_id=''):
    url = 'https://corpus.prozhito.org/api/notes/search'
    params = {
        'search_type': 'diaries',
        'offset': offset,
        'searchNotesSession': session_id
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Ошибка: {response.status_code}")
        return None


In [ ]:
def save_entries(entries, filename):
    # Проверка на существование файла и загрузка данных, если файл существует
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            existing_entries = json.load(f)
    except FileNotFoundError:
        existing_entries = {}

    # Объединяем существующие записи с новыми
    for year, notes in entries.items():
        if year not in existing_entries:
            existing_entries[year] = []
        existing_entries[year].extend(notes)

    # Сортируем записи по году
    sorted_entries = {year: existing_entries[year] for year in sorted(existing_entries.keys())}

    # Записываем объединенные записи обратно в файл
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(sorted_entries, f, ensure_ascii=False, indent=2)


In [ ]:
def main():
    all_entries = {}
    processed_links_file = "processed_diary_links (1).txt"

    # Загружаем уже обработанные ссылки
    try:
        with open(processed_links_file, "r", encoding="utf-8") as f:
            processed_links = set(line.strip() for line in f if line.strip())
    except FileNotFoundError:
        processed_links = set()

    # Открываем файл со ссылками на страницы людей
    with open("/content/diary_links.txt", "r", encoding="utf-8") as f:
        diary_urls = [line.strip() for line in f if line.strip()]

    for diary_url in diary_urls:
        if diary_url in processed_links:
            print(f"Ссылка уже обработана: {diary_url}, пропускаем...")
            continue

        print(f"Обрабатывается: {diary_url}")

        # Получаем корректную ссылку
        dynamic_link = get_dynamic_link(diary_url)
        if not dynamic_link:
            print("Не удалось найти корректную ссылку, пытаемся собирать данные напрямую.")

            # Если не нашли ссылку, используем вторичную функцию для сбора дневников
            entries, total_entries = scrape_diary_entries(diary_url)
            for year, notes in entries.items():
                if year not in all_entries:
                    all_entries[year] = []
                all_entries[year].extend(notes)
            print(f"Собрано {total_entries} записей напрямую")

            # Добавляем ссылку в обработанные
            with open(processed_links_file, "a", encoding="utf-8") as f:
                f.write(diary_url + "\n")

            continue

        print(f"Найдена ссылка: {dynamic_link}")

        # Для offset=0 используем Selenium и scrape_diary_entries()
        print("Обрабатываем offset=0 через Selenium...")
        entries, total_entries = scrape_diary_entries(diary_url)
        for year, notes in entries.items():
            if year not in all_entries:
                all_entries[year] = []
            all_entries[year].extend(notes)
        print(f"Собрано {total_entries} записей с offset=0.")

        # Получаем максимальный offset
        max_offset = get_max_offset(dynamic_link)
        if max_offset is None:
            print("Не удалось найти максимальный offset")
            continue

        print(f"Максимальное значение offset: {max_offset}")

        # Преобразуем ссылку в API ссылку
        api_url = convert_url(dynamic_link)
        if not api_url:
            print("Не удалось преобразовать ссылку в API.")
            continue

        print(f"API ссылка: {api_url}")

        # Получаем номер сессии API
        session_id = fetch_search_notes_session(api_url)
        if not session_id:
            print("Не удалось получить номер сессии")
            continue

        print(f"Номер сессии: {session_id}")

        # Обрабатываем offset > 0 через API
        for offset in range(25, max_offset + 1, 25):  # С шагом 25
            print(f"Запрашиваем API с offset={offset}...")
            entries = fetch_diary_entries(offset=offset, session_id=session_id)
            if entries:
                for note in entries["data"]["notes"]:
                    year = note["date"][:4]  # Извлекаем год из даты
                    if year not in all_entries:
                        all_entries[year] = []
                    all_entries[year].append(note["text"])

        # Добавляем ссылку в обработанные
        with open(processed_links_file, "a", encoding="utf-8") as f:
            f.write(diary_url + "\n")

    # Сохраняем результат
    save_entries(all_entries, "diary_entries_final (1).json")
    print(f"Финальные записи сохранены в 'diary_entries_final.json'.")

if __name__ == "__main__":
    main()

Ссылка уже обработана: https://corpus.prozhito.org/notes?date=%221941-01-01%22&diaries=%5B2892%5D, пропускаем...
Ссылка уже обработана: https://corpus.prozhito.org/notes?date=%221924-01-01%22&diaries=%5B1963%5D, пропускаем...
Ссылка уже обработана: https://corpus.prozhito.org/notes?date=%221941-01-01%22&diaries=%5B1343%5D, пропускаем...
Ссылка уже обработана: https://corpus.prozhito.org/notes?date=%221921-01-01%22&diaries=%5B1629%5D, пропускаем...
Ссылка уже обработана: https://corpus.prozhito.org/notes?date=%221917-01-01%22&diaries=%5B109%5D, пропускаем...
Ссылка уже обработана: https://corpus.prozhito.org/notes?date=%221941-01-01%22&diaries=%5B1972%5D, пропускаем...
Ссылка уже обработана: https://corpus.prozhito.org/notes?date=%221941-01-01%22&diaries=%5B5652%5D, пропускаем...
Ссылка уже обработана: https://corpus.prozhito.org/notes?date=%221967-01-01%22&diaries=%5B5450%5D, пропускаем...
Ссылка уже обработана: https://corpus.prozhito.org/notes?date=%221941-01-01%22&diaries=%5B1767%5D